In [ ]:
!pip install eli5==0.13.0
!apt-get install -y fonts-nanum

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import pickle
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

import eli5
from eli5.sklearn import PermutationImportance

# 데이터 불러오기
dt = pd.read_csv('train3.csv', encoding='utf-8')
dt_test = pd.read_csv('test3.csv')
dt_bus = pd.read_csv('bus_feature.csv')

# 반경 300m 내 버스 정류장 개수 계산 함수 정의
def count_nearby_bus_stops(x, y, bus_df, radius=300):
    distances = np.sqrt((bus_df['X좌표'] - x)**2 + (bus_df['Y좌표'] - y)**2)
    return (distances <= radius).sum()

# train과 test 데이터에 파생변수로 버스 정류장 개수 추가
dt['num_bus_stops_nearby'] = dt.apply(lambda row: count_nearby_bus_stops(row['좌표X'], row['좌표Y'], dt_bus, radius=300), axis=1)
dt_test['num_bus_stops_nearby'] = dt_test.apply(lambda row: count_nearby_bus_stops(row['좌표X'], row['좌표Y'], dt_bus, radius=300), axis=1)

# 전체 버스 정류장 개수를 train/test에 추가
bus_stop_count = dt_bus.shape[0]
dt['bus_stop_count'] = bus_stop_count
dt_test['bus_stop_count'] = bus_stop_count

# train/test 구분용 is_test 컬럼 추가 후 데이터 합치기
dt['is_test'] = 0
dt_test['is_test'] = 1
concat = pd.concat([dt, dt_test], ignore_index=True)

# 의미 없는 값들을 결측치로 처리
concat['등기신청일자'] = concat['등기신청일자'].replace(' ', np.nan)
concat['거래유형'] = concat['거래유형'].replace('-', np.nan)
concat['중개사소재지'] = concat['중개사소재지'].replace('-', np.nan)

# 결측치가 100만 개 이하인 컬럼만 선택
missing = concat.isnull().sum()
selected_cols = list(missing[missing <= 1000000].index)
concat_select = concat[selected_cols]

# '본번', '부번'을 문자열로 변환
concat_select['본번'] = concat_select['본번'].astype(str)
concat_select['부번'] = concat_select['부번'].astype(str)

# 연속형과 범주형 변수 분리
continuous_columns = [col for col in concat_select.columns if pd.api.types.is_numeric_dtype(concat_select[col])]
categorical_columns = [col for col in concat_select.columns if col not in continuous_columns]

# 범주형 변수 결측치는 'NULL'로 채우고, 연속형 변수는 선형 보간으로 결측치 처리
concat_select[categorical_columns] = concat_select[categorical_columns].fillna('NULL')
concat_select[continuous_columns] = concat_select[continuous_columns].interpolate(method='linear', axis=0)

# 이상치 제거 (IQR 방식) - '전용면적' 컬럼 기준
def remove_outliers_iqr(df, column):
    train_df = df.query('is_test == 0')
    test_df = df.query('is_test == 1')

    Q1 = train_df[column].quantile(0.25)
    Q3 = train_df[column].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    train_df = train_df[(train_df[column] >= lower_bound) & (train_df[column] <= upper_bound)]
    return pd.concat([train_df, test_df], ignore_index=True)

concat_select = remove_outliers_iqr(concat_select, '전용면적')

# '시군구'를 '구'와 '동'으로 분리하고 기존 '시군구' 컬럼 제거
concat_select['구'] = concat_select['시군구'].map(lambda x: x.split()[1])
concat_select['동'] = concat_select['시군구'].map(lambda x: x.split()[2])
concat_select.drop(columns=['시군구'], inplace=True)

# '계약년월'에서 '계약년'과 '계약월' 분리 후 기존 컬럼 제거
concat_select['계약년'] = concat_select['계약년월'].astype(str).str[:4]
concat_select['계약월'] = concat_select['계약년월'].astype(str).str[4:]
concat_select.drop(columns=['계약년월'], inplace=True)

# 강남 여부 파생 변수 생성
gangnam = ['강서구', '영등포구', '동작구', '서초구', '강남구', '송파구', '강동구']
concat_select['강남여부'] = concat_select['구'].apply(lambda x: 1 if x in gangnam else 0)

# 건축년도 기준 신축 여부 변수 생성 (2009년 이후 신축)
concat_select['신축여부'] = concat_select['건축년도'].apply(lambda x: 1 if x >= 2009 else 0)

# 다시 train과 test 데이터 분리 및 is_test 컬럼 제거
dt_train = concat_select.query('is_test == 0').drop(columns=['is_test'])
dt_test = concat_select.query('is_test == 1').drop(columns=['is_test'])

# test 데이터 타깃 컬럼 임시로 0으로 생성
dt_test['target'] = 0

# train 데이터의 연속형과 범주형 변수 분리
continuous_columns_v2 = [col for col in dt_train.columns if pd.api.types.is_numeric_dtype(dt_train[col])]
categorical_columns_v2 = [col for col in dt_train.columns if col not in continuous_columns_v2]

print("연속형 변수:", continuous_columns_v2)
print("범주형 변수:", categorical_columns_v2)

# 범주형 변수 레이블 인코딩 수행
label_encoders = {}
for col in tqdm(categorical_columns_v2):
    le = LabelEncoder()
    le.fit(dt_train[col].astype(str))
    dt_train[col] = le.transform(dt_train[col].astype(str))
    label_encoders[col] = le

    for label in np.unique(dt_test[col]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    dt_test[col] = le.transform(dt_test[col].astype(str))

# 타깃 분리 및 학습/검증 데이터 분리 (80:20 비율)
y_train = dt_train['target']
X_train = dt_train.drop(columns=['target'])
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=2023)

# XGBoost 모델 생성 및 학습
model = XGBRegressor(
    n_estimators=500,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
model.fit(X_train, y_train)

# 검증 데이터에 대해 예측 및 RMSE 평가
pred = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, pred))
print(f'Validation RMSE: {rmse:.2f}')

# 중요 변수 시각화
importances = pd.Series(model.feature_importances_, index=X_train.columns).sort_values(ascending=False)
plt.figure(figsize=(10,8))
sns.barplot(x=importances.values[:20], y=importances.index[:20])
plt.title("Top 20 Feature Importances")
plt.show()

# 학습된 모델 저장
with open('saved_model.pkl', 'wb') as f:
    pickle.dump(model, f)

# Permutation Importance 수행
perm = PermutationImportance(model, scoring="neg_mean_squared_error", random_state=42, n_iter=3)
perm.fit(X_val, y_val)
eli5.show_weights(perm, feature_names=X_val.columns.tolist())

# 테스트 데이터에서 타깃 제외 후 예측 수행
X_test = dt_test.drop(columns=['target'])
test_pred = model.predict(X_test)

# 제출 파일 생성
submission = pd.DataFrame({
    'id': dt_test['id'],
    'target': test_pred
})
submission.to_csv('submission.csv', index=False)


'apt-get'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
